In [16]:
import pandas as pd
import yaml
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [18]:
import os

os.getcwd()

'e:\\Videos\\My courses\\DEPI'

In [23]:
os.chdir('./Final Project')
os.getcwd()


'e:\\Videos\\My courses\\DEPI\\Final Project'

In [24]:
with open('configs/paths.yaml','r') as f:
    config = yaml.safe_load(f)

config


{'dir': {'data_dir': 'data',
  'artifacts_dir': 'artifacts',
  'raw_dir': 'data/raw',
  'processed_dir': 'data/processed',
  'models_dir': 'models',
  'config_dir': 'configs',
  'mlruns_dir': 'mlruns',
  'transformers_dir': 'transformers'},
 'paths': {'raw_data_path': 'data/raw/Superstore.csv',
  'processed_data_path': 'data/raw/sales time series.csv',
  'train_data_path': 'data/processed/train.csv',
  'test_data_path': 'data/processed/test.csv',
  'transformer': 'models/transformer.pkl',
  'rf_model_path': 'mlruns/750350709773541021/57e7048870224669874f8dd62775cce9/artifacts/models/model.pkl',
  'quantile_encoder_path': 'transformers/quantile_transformer.pkl',
  'label_encoder_path': 'transformers/label_encoder.pkl',
  'scaler_encoder_path': 'transformers/scaler.pkl'}}

In [25]:
data_path = config['paths']['processed_data_path']
df = pd.read_csv(data_path,encoding='ISO-8859-1')

df.head()

,Order Date,Ship Mode,Customer Name,Segment,City,State,Region,Category,Sub-Category,Product Name,...,Seasons_of_the_year,month,day_of_week,week_of_year,is_holiday,Sales,Quantity,Discount,Profit,price_per_unit
0,2013-11-09,Second Class,Claire Gute,Consumer,Henderson,Kentucky,South,Furniture,Bookcases,Bush Somerset Collection Bookcase,...,Autumn,11,5,45,False,261.9600,2,0.00,41.9136,130.9800
1,2013-11-09,Second Class,Claire Gute,Consumer,Henderson,Kentucky,South,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",...,Autumn,11,5,45,False,731.9400,3,0.00,219.5820,243.9800
2,2013-06-13,Second Class,Darrin Van Huff,Corporate,Los Angeles,California,West,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,...,Summer,6,3,24,False,14.6200,2,0.00,6.8714,7.3100
3,2012-10-11,Standard Class,Sean O'Donnell,Consumer,Fort Lauderdale,Florida,South,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,...,Autumn,10,3,41,False,957.5775,5,0.45,-383.0310,191.5155
4,2012-10-11,Standard Class,Sean O'Donnell,Consumer,Fort Lauderdale,Florida,South,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,...,Autumn,10,3,41,False,22.3680,2,0.20,2.5164,11.1840


In [26]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Order Date           9994 non-null   object 
 1   Ship Mode            9994 non-null   object 
 2   Customer Name        9994 non-null   object 
 3   Segment              9994 non-null   object 
 4   City                 9994 non-null   object 
 5   State                9994 non-null   object 
 6   Region               9994 non-null   object 
 7   Category             9994 non-null   object 
 8   Sub-Category         9994 non-null   object 
 9   Product Name         9994 non-null   object 
 10  Time_taken           9994 non-null   int64  
 11  Seasons_of_the_year  9994 non-null   object 
 12  month                9994 non-null   int64  
 13  day_of_week          9994 non-null   int64  
 14  week_of_year         9994 non-null   int64  
 15  is_holiday           9994 non-null   b

# prepare the data 

In [27]:
# Change data type for "Order Data" , "ship Date" to DateTime
df['Order Date'] = pd.to_datetime(df['Order Date'],format="%Y-%m-%d")

In [58]:
start_date = df['Order Date'].min()
end_date = df['Order Date'].max()

full_date_range = pd.date_range(start=start_date,end=end_date,freq='D')
full_date_df = pd.DataFrame( full_date_range ,columns=['Order Date'])

df_daily = df.groupby('Order Date').agg({'Sales':'sum',
                                   'Profit':'sum',
                                   'Quantity':'sum',
                                   'Discount':'sum'}).reset_index().sort_values(by='Order Date')

df_daily = pd.merge(full_date_df,df_daily,on='Order Date',how='left')


columns_to_fill = df_daily.columns[1:].to_list()

df_daily[columns_to_fill] = df_daily[columns_to_fill].fillna(0)

df_daily.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458 entries, 0 to 1457
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Order Date  1458 non-null   datetime64[ns]
 1   Sales       1458 non-null   float64       
 2   Profit      1458 non-null   float64       
 3   Quantity    1458 non-null   float64       
 4   Discount    1458 non-null   float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 57.1 KB


In [59]:
import holidays
from datetime import date

us_holidays = holidays.UnitedStates()

# Add holidays
print(date(2013,12 ,25) in us_holidays)
df_daily['is_holiday'] = df_daily['Order Date'].apply(lambda x : x in us_holidays).astype(int)

df_daily['year'] = df_daily['Order Date'].dt.year
df_daily['month'] = df_daily['Order Date'].dt.month
df_daily['day_of_year'] = df_daily['Order Date'].dt.dayofyear
df_daily['day_of_week'] = df_daily['Order Date'].dt.dayofweek
df_daily['is_weekend'] = df_daily['day_of_week'].isin([5, 6]).astype(int)
df_daily['quarter'] = df_daily['Order Date'].dt.quarter

# Create lag features for sales
df_daily['sales_lag_1'] = df_daily['Sales'].shift(1)  
df_daily['sales_lag_2'] = df_daily['Sales'].shift(2)
df_daily['sales_lag_3'] = df_daily['Sales'].shift(3)      # Previous day's sales
df_daily['sales_lag_7'] = df_daily['Sales'].shift(7)    # Sales from 1 week ago
df_daily['sales_lag_14'] = df_daily['Sales'].shift(14)  # Sales from 2 weeks ago
df_daily['sales_lag_30'] = df_daily['Sales'].shift(30)  # Sales from 1 month ago
df_daily['sales_lag_365'] = df_daily['Sales'].shift(365) # Sales from 1 year ago


df_daily['Profit_lag_7'] = df_daily['Profit'].shift(7) # profit from 2 weeks ago
df_daily['Quantity_lag_7'] = df_daily['Quantity'].shift(7) # Qunatity from 2 weeks ago 

# df_daily['Profit_lag_7'] = df_daily['Profit'].shift(7) # profit from 2 weeks ago
df_daily['Quantity_lag_14'] = df_daily['Quantity'].shift(14)
df_daily['Quantity_lag_30'] = df_daily['Quantity'].shift(30)


df_daily['rolling_mean_sales_3d'] = df_daily['Sales'].rolling(window=3).mean() # Calculate 3-day rolling mean of sales
df_daily['rolling_mean_sales_7d'] = df_daily['Sales'].rolling(window=7).mean()  # (7-day) rolling mean of sales
df_daily['rolling_mean_sales_14d'] = df_daily['Sales'].rolling(window=14).mean()
df_daily['rolling_mean_sales_30d'] = df_daily['Sales'].rolling(window=30).mean() #(30-day) rolling mean of sales
df_daily['rolling_mean_sales_1q'] = df_daily['Sales'].rolling(window=120).mean()

True


In [60]:
# Drop columns that are not needed for the model
df_daily.drop(['Discount','Profit','Quantity'],axis=1,inplace=True)
df_daily = df_daily.fillna(0)

In [61]:
df_daily.to_csv(config['dir']['processed_dir']+'/daily_data.csv')

In [74]:
# Set the Order Date as index for time series analysis
sales = df_daily.set_index('Order Date').resample('D').sum()

# Split data into training (up to Oct 1, 2014) and test (from Oct 2, 2014) sets
train_rf = sales.loc[:'2014-12-15']
test_rf  = sales.loc['2014-12-16':]

# Extract dates for later use
dates_test = test_rf.reset_index()['Order Date']
dates_train = train_rf.reset_index()['Order Date']

# Prepare features (X) by dropping target variable and date columns
x_train  = train_rf.reset_index().drop(['Sales','Order Date'],axis=True)
x_test  = test_rf.reset_index().drop(['Sales','Order Date'],axis=True)

# Prepare target variable (y) for both training and test sets
y_train = train_rf.reset_index()['Sales']
y_test = test_rf.reset_index()['Sales']

In [64]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1428 entries, 0 to 1427
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   is_holiday              1428 non-null   int32  
 1   year                    1428 non-null   int32  
 2   month                   1428 non-null   int32  
 3   day_of_year             1428 non-null   int32  
 4   day_of_week             1428 non-null   int32  
 5   is_weekend              1428 non-null   int32  
 6   quarter                 1428 non-null   int32  
 7   sales_lag_1             1428 non-null   float64
 8   sales_lag_2             1428 non-null   float64
 9   sales_lag_3             1428 non-null   float64
 10  sales_lag_7             1428 non-null   float64
 11  sales_lag_14            1428 non-null   float64
 12  sales_lag_30            1428 non-null   float64
 13  sales_lag_365           1428 non-null   float64
 14  Profit_lag_7            1428 non-null   

In [75]:
print('x_train shape: ',x_train.shape)
print('x_test shape: ',x_test.shape)
print('y_train shape: ',y_train.shape)
print('y_test shape: ',y_test.shape)
print('dates_train shape: ',dates_train.shape)
print('dates_test shape: ',dates_test.shape)


x_train shape:  (1442, 23)
x_test shape:  (16, 23)
y_train shape:  (1442,)
y_test shape:  (16,)
dates_train shape:  (1442,)
dates_test shape:  (16,)


# XGBoost

In [34]:
import xgboost as xgb
from xgboost import XGBRegressor
from xgboost import plot_importance
from sklearn.metrics import mean_squared_error, r2_score
import optuna
import mlflow


In [35]:
def plot_importance_features(model,booster):
    '''
  """
  Plots feature importance for an XGBoost model.

  Args:
  - model: A trained XGBoost model

    - booster: The type of booster to use. Can be 'gbtree' or 'gblinear'.
    '''
    
    fig,ax = plt.subplots(figsize=(12,6))

    xgb.plot_importance(model,
                    ax=ax)
    plt.title(f'Feature Importance ({booster})')
    plt.xlabel('Weight')
    plt.ylabel('Features')
    
    plt.tight_layout()
    plt.close()
    return fig


In [36]:
def get_or_create_experiment(experiment_name):
  """
  Retrieve the ID of an existing MLflow experiment or create a new one if it doesn't exist.

  This function checks if an experiment with the given name exists within MLflow.
  If it does, the function returns its ID. If not, it creates a new experiment
  with the provided name and returns its ID.

  Parameters:
  - experiment_name (str): Name of the MLflow experiment.

  Returns:
  - str: ID of the existing or newly created MLflow experiment.
  """

  if experiment := mlflow.get_experiment_by_name(experiment_name):
      return experiment.experiment_id
  else:
      return mlflow.create_experiment(experiment_name)

In [66]:
def plot_forecast(dates_test,y_test,y_pred):

    fig , ax = plt.subplots(figsize=(15,6))
    # Convert y_test and y_pred to pandas Series
    dates_test = pd.to_datetime(dates_test)

    y_test_series = pd.Series(y_test.values, index=dates_test)
    y_pred_series = pd.Series(y_pred, index=dates_test)

    y_test_resampled = y_test_series.resample('D').sum()
    y_pred_resampled = y_pred_series.resample('D').sum()

    ax.plot(y_test_resampled.index, y_test_resampled.values, label='Actual (Monthly)')
    ax.plot(y_pred_resampled.index, y_pred_resampled.values, label='XGBoost Forecast (Weekly)', linestyle='--')

    ax.set_xlabel('Date')
    ax.set_ylabel('Sales')
    ax.set_title('XGBoost Forecast vs Actual Sales (Optuna)')
    ax.legend()
    plt.close()
    return fig
    


In [76]:
dtrain = xgb.DMatrix(x_train,label=y_train)
dtest = xgb.DMatrix(x_test,label=y_test)

## XGBoost Optuna

In [77]:
def objective(trial):

      # Define hyperparameters
    params = {
          "objective": "reg:squarederror",
          "eval_metric": "rmse",
          "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
          "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
          "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
      }

    if params["booster"] == "gbtree" or params["booster"] == "dart":
          params["max_depth"] = trial.suggest_int("max_depth", 1, 9)
          params["eta"] = trial.suggest_float("eta", 1e-8, 1.0,)
          params["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0,)
          params["grow_policy"] = trial.suggest_categorical(
              "grow_policy", ["depthwise", "lossguide"]
          )

      # Train XGBoost model
    bst = xgb.train(params, dtrain)
    preds = bst.predict(dtest)
    error = mean_squared_error(y_test, preds)

    return error

In [78]:
study = optuna.create_study(direction='minimize')
study.optimize(objective,n_trials=1500,show_progress_bar=True)

[I 2025-05-29 18:28:20,178] A new study created in memory with name: no-name-d314687d-b0a1-464f-82ef-55398439ddaf


  0%|          | 0/1500 [00:00<?, ?it/s]

[I 2025-05-29 18:28:20,803] Trial 0 finished with value: 731415.0055457047 and parameters: {'booster': 'gbtree', 'lambda': 0.05795682693790605, 'alpha': 0.0006904633035343957, 'max_depth': 8, 'eta': 0.5098913392948707, 'gamma': 0.9560164165062883, 'grow_policy': 'lossguide'}. Best is trial 0 with value: 731415.0055457047.
[I 2025-05-29 18:28:20,897] Trial 1 finished with value: 1191412.7769699243 and parameters: {'booster': 'dart', 'lambda': 0.6879161956006257, 'alpha': 6.045171263995739e-06, 'max_depth': 8, 'eta': 0.6578322524300376, 'gamma': 0.06602587483035396, 'grow_policy': 'depthwise'}. Best is trial 0 with value: 731415.0055457047.
[I 2025-05-29 18:28:20,904] Trial 2 finished with value: 4446737581495.244 and parameters: {'booster': 'gblinear', 'lambda': 0.6844385551108894, 'alpha': 0.4762328100247797}. Best is trial 0 with value: 731415.0055457047.
[I 2025-05-29 18:28:20,941] Trial 3 finished with value: 2804021.7812774265 and parameters: {'booster': 'dart', 'lambda': 0.0063407

In [51]:
experiment_id = get_or_create_experiment(experiment_name='sales_forecast')
experiment_id

'278903003089258674'

In [79]:
mlflow.set_experiment(experiment_name='sales_forecast')
with mlflow.start_run(run_name='XGBoost'):
    mlflow.log_params(study.best_params)
    best_params = study.best_params

    best_model = xgb.train(best_params,dtrain)

    mlflow.xgboost.log_model(xgb_model=best_model,
                             artifact_path='models',
                             registered_model_name='xgb_model')

    y_pred = best_model.predict(dtest)

    mse_xgb = mean_squared_error(y_test,y_pred)
    rmse_xgb = np.sqrt(mse_xgb)
    r2_xgb = r2_score(y_test,y_pred)

    mlflow.log_metrics({'mse':mse_xgb,'rmse':rmse_xgb,'r2':r2_xgb})

    mlflow.set_tags({
        'model_type':'XGBoost',
        'experiment_id':experiment_id,
        'run_name':'XGBoost',
        'run_id':mlflow.active_run().info.run_id
    })

    fig_importance = plot_importance_features(best_model,booster='dart')
    mlflow.log_figure(fig_importance,artifact_file="feature_importances_xgb.png")

    fig_forecast = plot_forecast(dates_test,y_test,y_pred)
    mlflow.log_figure(fig_forecast,artifact_file="forecast_xgb.png")

    print('mse for xgb: ',mse_xgb)
    print('rmse for xgb: ',rmse_xgb)
    print('r2 for xgb: ',r2_xgb)

    mlflow.end_run()



c:\Users\Check-In\anaconda3\Lib\site-packages\mlflow\xgboost\__init__.py:168: UserWarning: [18:32:37] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  xgb_model.save_model(model_data_path)
2025/05/29 18:32:45 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'xgb_model' already exists. Creating a new version of this model...
Created version '3' of model 'xgb_model'.


mse for xgb:  60732.68745601273
rmse for xgb:  246.44002811234367
r2 for xgb:  0.984813805074074


In [80]:
print('Best trial:')
print(study.best_trial)

Best trial:
FrozenTrial(number=1480, state=1, values=[60732.68745601273], datetime_start=datetime.datetime(2025, 5, 29, 18, 32, 7, 757661), datetime_complete=datetime.datetime(2025, 5, 29, 18, 32, 7, 922202), params={'booster': 'dart', 'lambda': 0.01426367196283264, 'alpha': 2.557020077788047e-06, 'max_depth': 5, 'eta': 0.4237046876260822, 'gamma': 0.24470842918137692, 'grow_policy': 'depthwise'}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'booster': CategoricalDistribution(choices=('gbtree', 'gblinear', 'dart')), 'lambda': FloatDistribution(high=1.0, log=True, low=1e-08, step=None), 'alpha': FloatDistribution(high=1.0, log=True, low=1e-08, step=None), 'max_depth': IntDistribution(high=9, log=False, low=1, step=1), 'eta': FloatDistribution(high=1.0, log=False, low=1e-08, step=None), 'gamma': FloatDistribution(high=1.0, log=False, low=1e-08, step=None), 'grow_policy': CategoricalDistribution(choices=('depthwise', 'lossguide'))}, trial_id=1480, value=None)


# Random Forest 

In [81]:
from sklearn.ensemble import RandomForestRegressor
# Define objective function
def objective(trial):
    # Suggest values for hyperparameters
    params = {
    'n_estimators' : trial.suggest_int("n_estimators", 10, 200, log=True),
    'max_depth' : trial.suggest_int("max_depth", 2, 32),
    'min_samples_split' : trial.suggest_int("min_samples_split", 2, 10),
    'min_samples_leaf' :trial.suggest_int("min_samples_leaf", 1, 10)
    }
    # Create and fit random forest model
    model = RandomForestRegressor(**params)
    model.fit(x_train, y_train)

    # Make predictions and calculate RMSE
    y_pred = model.predict(x_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    r2 = r2_score(y_test, y_pred)

    # Return MAE
    return rmse

study = optuna.create_study(direction='minimize')

study.optimize(objective,n_trials=100,show_progress_bar=True)

[I 2025-05-29 18:32:55,545] A new study created in memory with name: no-name-0accb1f3-e454-465b-b127-0129905b2cf9


  0%|          | 0/100 [00:00<?, ?it/s]

[I 2025-05-29 18:32:57,794] Trial 0 finished with value: 490.6326142480323 and parameters: {'n_estimators': 174, 'max_depth': 31, 'min_samples_split': 5, 'min_samples_leaf': 8}. Best is trial 0 with value: 490.6326142480323.
[I 2025-05-29 18:32:58,670] Trial 1 finished with value: 521.9423777459169 and parameters: {'n_estimators': 66, 'max_depth': 11, 'min_samples_split': 4, 'min_samples_leaf': 5}. Best is trial 0 with value: 490.6326142480323.
[I 2025-05-29 18:32:59,012] Trial 2 finished with value: 577.2429449292449 and parameters: {'n_estimators': 34, 'max_depth': 6, 'min_samples_split': 5, 'min_samples_leaf': 4}. Best is trial 0 with value: 490.6326142480323.
[I 2025-05-29 18:32:59,296] Trial 3 finished with value: 558.5671925734089 and parameters: {'n_estimators': 25, 'max_depth': 31, 'min_samples_split': 3, 'min_samples_leaf': 10}. Best is trial 0 with value: 490.6326142480323.
[I 2025-05-29 18:33:01,107] Trial 4 finished with value: 516.6496927732886 and parameters: {'n_estimato

In [82]:
mlflow.set_experiment(experiment_name='sales_forecast')
with mlflow.start_run(run_name='Random Forest'):
    mlflow.log_params(study.best_params)
    best_params = study.best_params

    best_model = RandomForestRegressor(**best_params)
    best_model.fit(x_train,y_train)

    mlflow.sklearn.log_model(sk_model=best_model,
                             artifact_path='models',
                             registered_model_name='rf_model')
    
    y_pred = best_model.predict(x_test)

    mse_rf = mean_squared_error(y_test,y_pred)
    rmse_rf = np.sqrt(mse_rf)
    r2_rf = r2_score(y_test,y_pred)

    mlflow.log_metrics({'mse':mse_rf,'rmse':rmse_rf,'r2':r2_rf})

    mlflow.set_tags({
        'model_type':'Random Forest',
        'experiment_id':experiment_id,
        'run_name':'Random Forest',
        'run_id':mlflow.active_run().info.run_id
    })


    fig_forecast = plot_forecast(dates_test,y_test,y_pred)
    mlflow.log_figure(fig_forecast,artifact_file="forecast_rf.png")

    print('mse for rf: ',mse_rf)
    print('rmse for rf: ',rmse_rf)
    print('r2 for rf: ',r2_rf)

    mlflow.end_run()
    

2025/05/29 18:35:38 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'rf_model' already exists. Creating a new version of this model...
Created version '3' of model 'rf_model'.


mse for rf:  441912.0191422242
rmse for rf:  664.7646343949294
r2 for rf:  0.8894999983713227


In [83]:
!mlflow ui --backend-store-uri "file:///E:/Videos/My courses/DEPI/Final Project/mlruns" --port 5001

^C
